<a href="https://colab.research.google.com/github/devroopsaha744/toxicity-prediction-ai-bionnovate-Visonx.py/blob/main/IIT_Jodhpur_ML_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00


In [19]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 34.8 MB/s eta 0:00:00


In [44]:
from datasets import load_dataset

df = load_dataset("zpn/clintox")

#### **Loading the dependencies**

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs

In [62]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.metrics import binary_crossentropy
from sklearn.metrics import classification_report

### **Loading and preparing the data**


In [46]:
df

DatasetDict({
    train: Dataset({
        features: ['smiles', 'selfies', 'target'],
        num_rows: 1181
    })
    validation: Dataset({
        features: ['smiles', 'selfies', 'target'],
        num_rows: 148
    })
    test: Dataset({
        features: ['smiles', 'selfies', 'target'],
        num_rows: 148
    })
})

In [47]:
train = df['train']
test = df['test']
valid = df['validation']

train = pd.DataFrame(train)
test = pd.DataFrame(test)
valid = pd.DataFrame(valid)

In [48]:
train

,smiles,selfies,target
0,CC(C)C[C@H](NC(=O)CNC(=O)c1cc(Cl)ccc1Cl)B(O)O,[C][C][Branch1][C][C][C][C@H1][Branch2][Ring1]...,1
1,O=C(NCC(O)CO)c1c(I)c(C(=O)NCC(O)CO)c(I)c(N(CCO...,[O][=C][Branch1][=Branch2][N][C][C][Branch1][C...,0
2,Clc1cc(Cl)c(OCC#CI)cc1Cl,[Cl][C][=C][C][Branch1][C][Cl][=C][Branch1][=B...,0
3,N#Cc1cc(NC(=O)C(=O)[O-])c(Cl)c(NC(=O)C(=O)[O-])c1,[N][#C][C][=C][C][Branch1][O][N][C][=Branch1][...,0
4,NS(=O)(=O)c1cc(Cl)c(Cl)c(S(N)(=O)=O)c1,[N][S][=Branch1][C][=O][=Branch1][C][=O][C][=C...,0
...,...,...,...
1176,CC(C[NH2+]C1CCCCC1)OC(=O)c1ccccc1,[C][C][Branch1][O][C][NH2+1][C][C][C][C][C][C]...,0
1177,CC(C(=O)[O-])c1ccc(C(=O)c2cccs2)cc1,[C][C][Branch1][=Branch1][C][=Branch1][C][=O][...,0
1178,CC(c1cc2ccccc2s1)N(O)C(N)=O,[C][C][Branch1][=C][C][=C][C][=C][C][=C][C][=C...,0
1179,CC(O)C(CO)NC(=O)C1CSSCC(NC(=O)C([NH3+])Cc2cccc...,[C][C][Branch1][C][O][C][Branch1][Ring1][C][O]...,0


In [49]:
test

,smiles,selfies,target
0,C[NH+]1C[C@H](C(=O)N[C@]2(C)O[C@@]3(O)[C@@H]4C...,[C][NH1+1][C][C@H1][Branch2][Ring2][#C][C][=Br...,0
1,C[C@]1(Cn2ccnn2)[C@H](C(=O)[O-])N2C(=O)C[C@H]2...,[C][C@][Branch1][=Branch2][C][N][C][=C][N][=N]...,0
2,C[NH+]1CCC[C@@H]1CCO[C@](C)(c1ccccc1)c1ccc(Cl)cc1,[C][NH1+1][C][C][C][C@@H1][Ring1][Branch1][C][...,0
3,Nc1nc(NC2CC2)c2ncn([C@H]3C=C[C@@H](CO)C3)c2n1,[N][C][=N][C][Branch1][#Branch1][N][C][C][C][R...,0
4,OC[C@H]1O[C@@H](n2cnc3c2NC=[NH+]C[C@H]3O)C[C@@...,[O][C][C@H1][O][C@@H1][Branch1][S][N][C][=N][C...,0
...,...,...,...
143,O=C1O[C@H]([C@@H](O)CO)C([O-])=C1O,[O][=C][O][C@H1][Branch1][#Branch1][C@@H1][Bra...,0
144,C#CCC(Cc1cnc2nc(N)nc(N)c2n1)c1ccc(C(=O)N[C@@H]...,[C][#C][C][C][Branch2][Ring1][=Branch1][C][C][...,0
145,C#CC[NH2+][C@@H]1CCc2ccccc21,[C][#C][C][NH2+1][C@@H1][C][C][C][=C][C][=C][C...,0
146,[H]/[NH+]=C(\N)c1ccc(OCCCCCOc2ccc(/C(N)=[NH+]/...,[H][/NH1+1][=C][Branch1][C][\N][C][=C][C][=C][...,0


In [50]:
valid

,smiles,selfies,target
0,CC(C)OC(=O)CCC/C=C\C[C@@H]1[C@@H](/C=C/[C@@H](...,[C][C][Branch1][C][C][O][C][=Branch1][C][=O][C...,0
1,CC(C)Nc1cccnc1N1CCN(C(=O)c2cc3cc(NS(C)(=O)=O)c...,[C][C][Branch1][C][C][N][C][=C][C][=C][N][=C][...,0
2,CC(C)n1c(/C=C/[C@H](O)C[C@H](O)CC(=O)[O-])c(-c...,[C][C][Branch1][C][C][N][C][Branch2][Ring1][C]...,0
3,CC(C)COCC(CN(Cc1ccccc1)c1ccccc1)[NH+]1CCCC1,[C][C][Branch1][C][C][C][O][C][C][Branch2][Rin...,0
4,CSCC[C@H](NC(=O)[C@H](Cc1c[nH]c2ccccc12)NC(=O)...,[C][S][C][C][C@H1][Branch2][Ring2][#Branch2][N...,0
...,...,...,...
143,C[C@H](OC(=O)c1ccccc1)C1=CCC23OCC[NH+](C)CC12C...,[C][C@H1][Branch1][=C][O][C][=Branch1][C][=O][...,0
144,C[C@@H](c1ncncc1F)[C@](O)(Cn1cncn1)c1ccc(F)cc1F,[C][C@@H1][Branch1][#Branch2][C][=N][C][=N][C]...,0
145,CC(C)C[C@@H](NC(=O)[C@H](C)NC(=O)CNC(=O)[C@@H]...,[C][C][Branch1][C][C][C][C@@H1][Branch2][Ring1...,0
146,C[C@H](O)[C@H](O)[C@H]1CNc2[nH]c(N)nc(=O)c2N1,[C][C@H1][Branch1][C][O][C@H1][Branch1][C][O][...,0


In [51]:
train = train.drop(['selfies'], axis = 1) #We don't require this column

In [52]:
#generating and extracting features from the
train['mol'] = train['smiles'].apply(Chem.MolFromSmiles)
train['mol_wt'] = train['mol'].apply(lambda x: Descriptors.MolWt(x) if x is not None else None)
train['logp'] = train['mol'].apply(lambda x: Descriptors.MolLogP(x) if x is not None else None)
train['tpsa'] = train['mol'].apply(lambda x: Descriptors.TPSA(x) if x is not None else None)
train['hbd'] = train['mol'].apply(lambda x: Descriptors.NumHDonors(x) if x is not None else None)
train['hba'] = train['mol'].apply(lambda x: Descriptors.NumHAcceptors(x) if x is not None else None)
train['rot_bonds'] = train['mol'].apply(lambda x: Descriptors.NumRotatableBonds(x) if x is not None else None)
train['ring_count'] = train['mol'].apply(lambda x: Descriptors.RingCount(x) if x is not None else None)
train['aromaticity'] = train['mol'].apply(lambda x: Descriptors.NumAromaticRings(x) if x is not None else None)

In [53]:
toxicophores = {
    'Aldehydes_Ketones': 'C(=O)',
    'Organophosphates': 'P(=O)(OCC)OCC',
    'Isocyanates': 'C(=O)N=C=O',
    'Heavy_Metals_Mercury': '[Hg]',
    'Aromatic_Hydrocarbons': 'c1ccccc1',
    'Nitriles': 'CC#N',
    'PAHs': 'c1ccc2cc3ccccc3cc2c1',
    'Ethers': 'CCOCC',
    'Azides': 'CC(=O)N=[N+]=[N-]',
    'Halogens_Chlorine': 'Cl'
}

# Create binary columns for each toxicophore
for toxicophore_name, smart_pattern in toxicophores.items():
    train[toxicophore_name] = train['mol'].apply(lambda x: 1 if x is not None and x.HasSubstructMatch(Chem.MolFromSmarts(smart_pattern)) else 0)

In [54]:
train

,smiles,target,mol,mol_wt,logp,tpsa,hbd,hba,rot_bonds,ring_count,...,Aldehydes_Ketones,Organophosphates,Isocyanates,Heavy_Metals_Mercury,Aromatic_Hydrocarbons,Nitriles,PAHs,Ethers,Azides,Halogens_Chlorine
0,CC(C)C[C@H](NC(=O)CNC(=O)c1cc(Cl)ccc1Cl)B(O)O,1,<rdkit.Chem.rdchem.Mol object at 0x78bd51bcd310>,361.034,1.26610,98.66,4,4,7,1,...,1,0,0,0,1,0,0,0,0,1
1,O=C(NCC(O)CO)c1c(I)c(C(=O)NCC(O)CO)c(I)c(N(CCO...,0,<rdkit.Chem.rdchem.Mol object at 0x78bd51bcda80>,807.114,-2.01600,199.89,8,9,12,1,...,1,0,0,0,1,0,0,0,0,0
2,Clc1cc(Cl)c(OCC#CI)cc1Cl,0,<rdkit.Chem.rdchem.Mol object at 0x78bd51bcc6d0>,361.393,4.42150,9.23,0,1,2,1,...,0,0,0,0,1,0,0,0,0,1
3,N#Cc1cc(NC(=O)C(=O)[O-])c(Cl)c(NC(=O)C(=O)[O-])c1,0,<rdkit.Chem.rdchem.Mol object at 0x78bd51bcd150>,309.621,-2.41152,162.25,2,7,2,1,...,1,0,0,0,1,0,0,0,0,1
4,NS(=O)(=O)c1cc(Cl)c(Cl)c(S(N)(=O)=O)c1,0,<rdkit.Chem.rdchem.Mol object at 0x78bd51bcd850>,305.164,0.28820,120.32,2,4,2,1,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,CC(C[NH2+]C1CCCCC1)OC(=O)c1ccccc1,0,<rdkit.Chem.rdchem.Mol object at 0x78bd504398c0>,262.373,2.12800,42.91,1,2,5,2,...,1,0,0,0,1,0,0,0,0,0
1177,CC(C(=O)[O-])c1ccc(C(=O)c2cccs2)cc1,0,<rdkit.Chem.rdchem.Mol object at 0x78bd50439930>,259.306,1.83250,57.20,0,4,4,2,...,1,0,0,0,1,0,0,0,0,0
1178,CC(c1cc2ccccc2s1)N(O)C(N)=O,0,<rdkit.Chem.rdchem.Mol object at 0x78bd504399a0>,236.296,2.73220,66.56,2,3,2,2,...,1,0,0,0,1,0,0,0,0,0
1179,CC(O)C(CO)NC(=O)C1CSSCC(NC(=O)C([NH3+])Cc2cccc...,0,<rdkit.Chem.rdchem.Mol object at 0x78bd50439a10>,1021.277,-2.23900,335.46,13,12,17,5,...,1,0,0,0,1,0,0,0,0,0


In [55]:
train =  train.drop(['smiles', 'mol'], axis = 1)

In [56]:
#Same preprocessing for test set
test = test.drop(['selfies'], axis = 1)
test['mol'] = test['smiles'].apply(Chem.MolFromSmiles)
test['mol_wt'] = test['mol'].apply(lambda x: Descriptors.MolWt(x) if x is not None else None)
test['logp'] = test['mol'].apply(lambda x: Descriptors.MolLogP(x) if x is not None else None)
test['tpsa'] = test['mol'].apply(lambda x: Descriptors.TPSA(x) if x is not None else None)
test['hbd'] = test['mol'].apply(lambda x: Descriptors.NumHDonors(x) if x is not None else None)
test['hba'] = test['mol'].apply(lambda x: Descriptors.NumHAcceptors(x) if x is not None else None)
test['rot_bonds'] = test['mol'].apply(lambda x: Descriptors.NumRotatableBonds(x) if x is not None else None)
test['ring_count'] = test['mol'].apply(lambda x: Descriptors.RingCount(x) if x is not None else None)
test['aromaticity'] = test['mol'].apply(lambda x: Descriptors.NumAromaticRings(x) if x is not None else None)

for toxicophore_name, smart_pattern in toxicophores.items():
    test[toxicophore_name] = test['mol'].apply(lambda x: 1 if x is not None and x.HasSubstructMatch(Chem.MolFromSmarts(smart_pattern)) else 0)

test =  test.drop(['smiles', 'mol'], axis = 1)

In [57]:
#Same preprocessing for valid set
valid = valid.drop(['selfies'], axis = 1)
valid['mol'] = valid['smiles'].apply(Chem.MolFromSmiles)
valid['mol_wt'] = valid['mol'].apply(lambda x: Descriptors.MolWt(x) if x is not None else None)
valid['logp'] = valid['mol'].apply(lambda x: Descriptors.MolLogP(x) if x is not None else None)
valid['tpsa'] = valid['mol'].apply(lambda x: Descriptors.TPSA(x) if x is not None else None)
valid['hbd'] = valid['mol'].apply(lambda x: Descriptors.NumHDonors(x) if x is not None else None)
valid['hba'] = valid['mol'].apply(lambda x: Descriptors.NumHAcceptors(x) if x is not None else None)
valid['rot_bonds'] = valid['mol'].apply(lambda x: Descriptors.NumRotatableBonds(x) if x is not None else None)
valid['ring_count'] = valid['mol'].apply(lambda x: Descriptors.RingCount(x) if x is not None else None)
valid['aromaticity'] = valid['mol'].apply(lambda x: Descriptors.NumAromaticRings(x) if x is not None else None)

for toxicophore_name, smart_pattern in toxicophores.items():
    valid[toxicophore_name] = valid['mol'].apply(lambda x: 1 if x is not None and x.HasSubstructMatch(Chem.MolFromSmarts(smart_pattern)) else 0)

valid =  valid.drop(['smiles', 'mol'], axis = 1)

In [64]:
x_train = train.drop(['target'], axis = 1 )
y_train = train['target']

x_valid = valid.drop(['target'], axis = 1)
y_valid = valid['target']

x_test = test.drop(['target'], axis =1)
y_test = test['target']

## **Exploring the Data**

# **Model Developement**


In [67]:
clf1 = LogisticRegression()
clf2 = SVC()
clf3 = KNeighborsClassifier()
clf4 = DecisionTreeClassifier()
clf5 = RandomForestClassifier()
clf6 = AdaBoostClassifier()
clf7 = GradientBoostingClassifier()
clf8 = XGBClassifier()

classifiers = [clf1, clf2, clf3, clf4, clf5, clf6, clf7, clf8]

for var in classifiers:
  var.fit(x_train, y_train)
  y_pred = var.predict(x_valid)

  print("\n")
  print(f"The Classification report of {var} is: \n {classification_report(y_valid, y_pred)} ")
  print("\n")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels wit



The Classification report of LogisticRegression() is: 
               precision    recall  f1-score   support

           0       0.96      1.00      0.98       141
           1       1.00      0.14      0.25         7

    accuracy                           0.96       148
   macro avg       0.98      0.57      0.61       148
weighted avg       0.96      0.96      0.94       148
 




The Classification report of SVC() is: 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98       141
           1       0.00      0.00      0.00         7

    accuracy                           0.95       148
   macro avg       0.48      0.50      0.49       148
weighted avg       0.91      0.95      0.93       148
 




The Classification report of KNeighborsClassifier() is: 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98       141
           1       0.00      0.00      0.00         7

    accuracy   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))




The Classification report of RandomForestClassifier() is: 
               precision    recall  f1-score   support

           0       0.95      0.99      0.97       141
           1       0.00      0.00      0.00         7

    accuracy                           0.95       148
   macro avg       0.48      0.50      0.49       148
weighted avg       0.91      0.95      0.93       148
 




The Classification report of AdaBoostClassifier() is: 
               precision    recall  f1-score   support

           0       0.95      0.97      0.96       141
           1       0.00      0.00      0.00         7

    accuracy                           0.93       148
   macro avg       0.48      0.49      0.48       148
weighted avg       0.91      0.93      0.92       148
 




The Classification report of GradientBoostingClassifier() is: 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98       141
           1       0.00      0.00      0.00  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))




The Classification report of XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...) is: 
               precision    recall  f1-score   support

           0       0.95      0.98      0.97       141
           1       0.00      0.00      0.00         7

    accuracy                           0.